In [1]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import FileUpload, SelectMultiple, Button, VBox, Output, Layout
from io import BytesIO
import mplcursors  # Import for interactive data point inspection
import os
import xlsxwriter  # Make sure this library is installed for XLSX support
import webbrowser
import tkinter as tk
from tkinter import filedialog

# Ensure interactive mode is on for Jupyter Notebook, remove this line if not using Jupyter
%matplotlib widget

# Set the directory where files will be saved
output_dir = 'C:\\Users\\DEXXMB39\\OneDrive - Jungheinrich AG\\Dokumente'
os.chdir(output_dir)  # Change to the output directory

# Global variables
df = None
fig = None  # Declare fig globally

# Function to be executed when the process button is clicked
def process_upload(b):
    global df
    if upload.value:
        file_info = next(iter(upload.value))
        delimiters = [',', ';', '\t', '|', ' ']
        file_read_success = False
        try:
            content = file_info['content']
            for sep in delimiters:
                try:
                    df = pd.read_csv(BytesIO(content), sep=sep, low_memory=False)
                    for col in df.columns:
                        df[col] = pd.to_numeric(df[col], errors='coerce')
                    if df is not None and len(df.columns) > 1 and not df.dropna(how='all').empty:
                        file_read_success = True
                        break
                except Exception:
                    continue

            if file_read_success:
                select_y.options = df.columns
                print("Please select parameters to plot.")
            else:
                print("File format not recognized or not enough parameters to plot.")
        except Exception as e:
            print("Error processing file:", e)

# Function to draw the graph based on selected parameters
def draw_graph(b):
    global output_area, df, fig  # Declare fig globally within this function
    with output_area:
        output_area.clear_output(wait=True)
        if df is not None and select_y.value:
            plt.close('all')
            fig, ax1 = plt.subplots(figsize=(10, 6))
            lines = []
            first_param = select_y.value[0]
            line1, = ax1.plot(df[df.columns[0]], df[first_param], color='b', label=first_param)
            lines.append(line1)
            ax1.set_xlabel('Time')
            ax1.set_ylabel(first_param, color='b')
            ax1.tick_params(axis='y', labelcolor='b')
            ax2 = None
            if len(select_y.value) > 1:
                ax2 = ax1.twinx()
                for param in select_y.value[1:]:
                    line, = ax2.plot(df[df.columns[0]], df[param], label=param)
                    lines.append(line)
                    if len(select_y.value) < 3:  # Set label for ax2 only if fewer than three parameters are selected
                        ax2.set_ylabel(param)
                    ax2.tick_params(axis='y')

            if len(select_y.value) >= 3:
                # Set the same y-axis limits for both ax1 and ax2 when three or more parameters are selected
                y_min = min(ax1.get_ylim()[0], ax2.get_ylim()[0])
                y_max = max(ax1.get_ylim()[1], ax2.get_ylim()[1])
                ax1.set_ylim(y_min, y_max)
                ax2.set_ylim(y_min, y_max)
                ax2.set_ylabel('')  # Remove the right vertical axis title when three or more parameters are selected

            if lines:
                labels_all = [line.get_label() for line in lines]
                ax1.legend(lines, labels_all, loc='upper left', fancybox=True, shadow=True)
            plt.title('Parameters over Time')
            mplcursors.cursor(lines, hover=True)
            plt.show()
        else:
            print("Please upload a file and select parameters to plot.")


# Function to save the graph as PNG
def save_as_png(b):
    global fig
    if fig:
        root = tk.Tk()
        root.withdraw()  # Hide the main window.
        file_path = filedialog.asksaveasfilename(defaultextension=".png",
                                                 filetypes=[("PNG files", "*.png")],
                                                 title="Save the graph as PNG")
        if file_path:  # Check if a file path was selected.
            fig.savefig(file_path)
            print(f"Graph saved as {file_path}")
        else:
            print("Save operation cancelled.")
        root.destroy()

# Function to save the graph as XLSX
def save_as_xlsx(b):
    global df, select_y
    # Check if data and parameters have been selected
    if df is not None and select_y.value:
        root = tk.Tk()
        root.withdraw()  # Hide the main Tkinter window
        # Get the file path for saving from the user
        file_path = filedialog.asksaveasfilename(
            defaultextension=".xlsx",
            filetypes=[("Excel files", "*.xlsx")],
            title="Save the data as XLSX"
        )

        if file_path:  # Proceed if a file path was provided
            with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
                selected_columns = [df.columns[0]] + list(select_y.value)
                df_to_save = df[selected_columns].copy()
                for col in selected_columns[1:]:
                    df_to_save[col] = pd.to_numeric(df_to_save[col], errors='coerce')
                df_to_save.to_excel(writer, sheet_name='Sheet1', startrow=0, header=True, index=False)

                workbook = writer.book
                worksheet = writer.sheets['Sheet1']

                chart = workbook.add_chart({'type': 'scatter', 'subtype': 'straight_with_markers'})

                num_rows = len(df_to_save) + 1
                for i, column in enumerate(selected_columns[1:], start=1):
                    chart.add_series({
                        'name':       ['Sheet1', 0, i],
                        'categories': ['Sheet1', 1, 0, num_rows - 1, 0],
                        'values':     ['Sheet1', 1, i, num_rows - 1, i],
                        'marker':     {'type': 'circle', 'size': 5},
                        'y2_axis':    True if i > 1 else False,
                    })

                # Remove the chart title
                chart.set_title({'none': True})

                # Set chart size (default size is usually 480x288 in Excel)
                # To triple the size, we adjust both width and height
                chart.set_size({'width': 1440, 'height': 864})  # Triple the default size

                x_axis_title = df.columns[0]
                primary_y_axis_title = selected_columns[1] if len(selected_columns) > 1 else 'Value'
                secondary_y_axis_title = selected_columns[2] if len(selected_columns) > 2 else 'Secondary Value'

                chart.set_x_axis({'name': x_axis_title})
                chart.set_y_axis({'name': primary_y_axis_title})
                if len(selected_columns) > 2:
                    chart.set_y2_axis({'name': secondary_y_axis_title})

                worksheet.insert_chart('G2', chart)

                print(f"Data saved as {file_path}")

            # Open the folder containing the saved file
            folder_path = os.path.dirname(file_path)
            webbrowser.open(f'file://{folder_path}')  # Open the folder in the default file manager
        else:
            print("Save operation cancelled.")

        root.destroy()  # Close the Tkinter window after operation
    else:
        print("No data to save or no parameters selected.")


# Create widgets
upload = FileUpload(accept='.csv', multiple=False)
process_button = Button(description="Process Uploaded File")
process_button.on_click(process_upload)
select_y = SelectMultiple(
    options=[], 
    description='Parameters:',
    disabled=False,
    layout=Layout(width='50%', height='120px')
)
draw_button = Button(description='Draw Graph')
draw_button.on_click(draw_graph)
save_png_button = Button(description='Save as PNG')
save_png_button.on_click(save_as_png)
save_xlsx_button = Button(description='Save as XLSX')
save_xlsx_button.on_click(save_as_xlsx)
output_area = Output()

# Display the widgets
display(VBox([upload, process_button, select_y, draw_button, save_png_button, save_xlsx_button, output_area]))
